# Data Processing

This notebook outlines the process of transforming raw JSON data from the `data/raw/` directory into structured SQLite tables for efficient analysis and visualization.

### Objective

The goal of this notebook is to:
- Read JSON files containing playlist, track, and genre data.
- Transform the raw data into Pandas DataFrames for preprocessing.
- Design a relational database schema with at least two tables linked by a foreign key.
- Store the preprocessed data in a SQLite database for further use.

## 1. Preparing the Environment

To begin data processing, the necessary libraries are imported, and the file paths for raw data and the SQLite database are defined.

- **Libraries**:  
  The following libraries are imported:
  - `os` and `json` for working with the file system and parsing JSON data.
  - `pandas` for data manipulation and transformation.
  - `sqlite3` for creating and interacting with the SQLite database.

- **File Paths**:  
  The raw data is located in the `../data/raw/` directory, and the processed database will be saved as `../data/processed_playlists.db`.

In [1]:
# Library imports
import os
import json
import pandas as pd
import sqlite3

In [ ]:
# Define file paths
raw_data_dir = "../data/raw/"
base_dir = os.path.abspath("../data") # os.path.abspath is used to get the absolute path of the directory
database_path = os.path.join(base_dir, "processed_playlists.db")

## 2. Data Processing

- **Input Data**:
  - Raw JSON files stored in the `..data/raw/` directory.
  - Each file corresponds to a specific playlist, containing:
    - Playlist metadata (e.g., ID, name, followers, etc.).
    - Track-level data (e.g., track name, artist, etc.).
    - Genre-level data for each track's artist.

- **Processing Steps**:
  - Read JSON files and load them into Pandas DataFrames.
  - Validate and clean the data, ensuring no missing or inconsistent fields.
  - Normalize nested JSON structures (e.g., extract artist and genre data into separate DataFrames).
  - Establish relationships between tables using foreign keys.

- **Output Data**:
  - SQLite database stored in the `data/` directory, containing the following tables:
    - **playlists**: Playlist-level metadata such as playlist ID, name, description, followers, and time of day.
    - **tracks**: Track-level metadata, including track ID, name, playlist ID, and genres.
    - **artists**: Artist-level metadata, including artist ID and name.
    - **track_artist**: A table linking tracks to their respective artists via foreign keys.

In [ ]:
# Get all JSON files in directory
files = [f for f in os.listdir(raw_data_dir) if f.endswith('.json')]

# Initialize empty lists for DataFrames
playlists_data = []
tracks_data = []
artists_data = []
track_artist_data = []

# Load and process each JSON file
for file in files:
    with open(os.path.join(raw_data_dir, file), 'r') as f:
        data = json.load(f)

        playlist_name = data['playlist_details'].get('name', '').lower()
        playlist_description = data['playlist_details'].get('description', '').lower()
        combined_text = f"{playlist_name} {playlist_description}"

        # Determine time of day based on the text in the playlist title / description
        if 'morning' in combined_text:
            time_of_day = 'morning'
        elif 'afternoon' in combined_text:
            time_of_day = 'afternoon'
        elif 'evening' in combined_text:
            time_of_day = 'evening'
        elif 'night' in combined_text:
            time_of_day = 'night'
        else:
            time_of_day = 'unknown'
        
        # Add playlist info
        playlist_info = {
            'playlist_id': data['playlist_id'],
            'name': data['playlist_details'].get('name', ''),  # Playlist name
            'description': data['playlist_details'].get('description', ''),  # Playlist description
            'followers': data['playlist_details'].get('followers', {}).get('total', 0),  # Number of followers
            'time_of_day': time_of_day  # Time of day, based on filename
        }
        playlists_data.append(playlist_info)
        
        # Process tracks
        for item in data['playlist_tracks']['items']:
            track = item['track']
            
            # Get genres if available
            track_genres = []
            if 'track_genres' in data:
                # Use the track name as the key to fetch genres
                track_genres = data['track_genres'].get(track['name'], [])
            else:
                print("No genres found for track: ", track['name'])
            
            track_info = {
                'track_id': track['id'],
                'name': track['name'],
                'playlist_id': data['playlist_id'],
                'genres': ','.join(track_genres) if track_genres else None # Convert list to comma-separated string
            }
            tracks_data.append(track_info)

            # Process artists
            for artist in track['artists']:
                # Add artist info
                artist_info = {
                    'artist_id': artist['id'],
                    'name': artist['name']
                }
                artists_data.append(artist_info)
                
                # Add track-artist relationship
                track_artist_info = {
                    'track_id': track['id'],  # Foreign key to Tracks
                    'artist_id': artist['id']  # Foreign key to Artists
                }
                track_artist_data.append(track_artist_info)

# Create DataFrames
playlist_df = pd.DataFrame(playlists_data)
tracks_df = pd.DataFrame(tracks_data)
artists_df = pd.DataFrame(artists_data).drop_duplicates(subset='artist_id')  # Remove duplicates
track_artist_df = pd.DataFrame(track_artist_data)

# Display the first few rows of each DataFrame
print("Playlist DataFrame:")
display(playlist_df.head())
print("\nTracks DataFrame:")
display(tracks_df.head())
print("\nArtists DataFrame:")
display(artists_df.head())
print("\nTrack-Artist Relationship DataFrame:")
display(track_artist_df.head())

Playlist DataFrame:


,playlist_id,name,description,followers,time_of_day
0,0e7BUrGydQFnw8WSpUoBjw,Night night music,,68,night
1,68JXTKfqFZEWO1DQRdVndh,Sleep Playlist😴💤,if ur not careful this playlist will put you t...,126112,unknown
2,25bMInOJfqU8nXEnzPSNSa,Chilled Summer Afternoon Vibes,,3711,afternoon
3,7eJQG4OIFTd5KsFua7IEBj,Sunday Chill ☕ Morning Playlist :-),Sunday vibe playlist for a lazy Sunday!,94687,morning
4,63sCiH22a44UxGge1gTuPi,Chill Morning 🥐☕ Breakfast Music 2024,Start your morning with soulful and chilled mu...,28727,morning



Tracks DataFrame:


,track_id,name,playlist_id,genres
0,3b16Un9e7LPQ61fevkpeRb,Shallow,0e7BUrGydQFnw8WSpUoBjw,classify
1,5TnQHTgPOBeuNC640bu3iG,7 Rings,0e7BUrGydQFnw8WSpUoBjw,classify
2,7nDCuwqvFxtn6Up1hSih2H,Can You Feel the Love Tonight,0e7BUrGydQFnw8WSpUoBjw,piano cover
3,7p32LHQj8QFSwJ3kFfT3Xe,"Nemo Egg (Main Title) - From ""Finding Nemo""",0e7BUrGydQFnw8WSpUoBjw,"orchestral soundtrack,soundtrack"
4,6T3Y2tPSd6zQYufcAkq66B,Once Upon a Dream,0e7BUrGydQFnw8WSpUoBjw,sleep



Artists DataFrame:


,artist_id,name
0,4duJIuAjkc0AmqHmpC3uOS,Cole Albrighton
1,6dBIk14xcZqU0cRsyjXNkX,Tyler and Mark
2,4Xx6QMLTWppMwdABkN0Afj,Piano Tribute Players
3,1csBgT42N4pPPs1HJhxXIK,Thomas Newman
4,0PS6tNISpLo5tFSH06JA5h,Relax α Wave



Track-Artist Relationship DataFrame:


,track_id,artist_id
0,3b16Un9e7LPQ61fevkpeRb,4duJIuAjkc0AmqHmpC3uOS
1,5TnQHTgPOBeuNC640bu3iG,6dBIk14xcZqU0cRsyjXNkX
2,7nDCuwqvFxtn6Up1hSih2H,4Xx6QMLTWppMwdABkN0Afj
3,7p32LHQj8QFSwJ3kFfT3Xe,1csBgT42N4pPPs1HJhxXIK
4,6T3Y2tPSd6zQYufcAkq66B,0PS6tNISpLo5tFSH06JA5h


In [ ]:
# Connect to SQLite database
conn = sqlite3.connect(database_path)

# Save DataFrames to database
try:
    playlist_df.to_sql('playlists', conn, if_exists='replace', index=False)
    print("playlists table saved successfully.")
except Exception as e:
    print(f"Error saving playlists table: {e}")

try:
    tracks_df.to_sql('tracks', conn, if_exists='replace', index=False)
    print("tracks table saved successfully.")
except Exception as e:
    print(f"Error saving tracks table: {e}")

try:
    artists_df.to_sql('artists', conn, if_exists='replace', index=False)
    print("artists table saved successfully.")
except Exception as e:
    print(f"Error saving artists table: {e}")

try:
    track_artist_df.to_sql('track_artist', conn, if_exists='replace', index=False)
    print("track_artist table saved successfully.")
except Exception as e:
    print(f"Error saving track_artist table: {e}")

print("Saving database to:", os.path.abspath(database_path))

# Close connection
conn.close()

playlists table saved successfully.
tracks table saved successfully.
artists table saved successfully.
track_artist table saved successfully.
Saving database to: /Users/miajaenike/Desktop/DS105A_Summative_2/data/processed_playlists.db
